In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import random
import torch
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
torch.cuda.device_count()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import torch
print(torch.cuda.is_available())

# Loading the Dataset

In [ ]:
# Paths for data.
Ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
Crema = "/kaggle/input/cremad/AudioWAV/"
Tess = "/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
Savee = "/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
# IEMOCAP = ""

## TESS

In [ ]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])

In [ ]:
# Joining them both
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

In [ ]:
sns.countplot(data=Tess_df, x='Emotions')

In [ ]:
# Sample Audio
path = np.array(Tess_df.Path)[1]
data, sample_rate = librosa.load(path)

plt.figure(figsize=(14,4))
librosa.display.waveshow(y=data, sr=sample_rate)
Audio(path)

## Ravdess

In [ ]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])

In [ ]:
# Joining them both
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

In [ ]:
sns.countplot(data=Ravdess_df, x='Emotions')

In [ ]:
# Sample Audio
path = np.array(Ravdess_df.Path)[1]
data, sample_rate = librosa.load(path)

plt.figure(figsize=(14,4))
librosa.display.waveshow(y=data, sr=sample_rate)
Audio(path)

## Crema-D

In [ ]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])

In [ ]:
# Joining them both
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

In [ ]:
sns.countplot(data=Crema_df, x='Emotions')

In [ ]:
# Sample Audio
path = np.array(Crema_df.Path)[1]
data, sample_rate = librosa.load(path)

plt.figure(figsize=(14,4))
librosa.display.waveshow(y=data, sr=sample_rate)
Audio(path)

## Savee

In [ ]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(Savee + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])

In [ ]:
# Joining them both
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

In [ ]:
sns.countplot(data=Savee_df, x='Emotions')

In [ ]:
# Sample Audio
path = np.array(Savee_df.Path)[1]
data, sample_rate = librosa.load(path)

plt.figure(figsize=(14,4))
librosa.display.waveshow(y=data, sr=sample_rate)
Audio(path)

# Combining All

In [ ]:
# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.shape

In [ ]:
data_path.head()

In [ ]:
sns.countplot(data=data_path, x='Emotions')

In [ ]:
data_path['Emotions'].value_counts()

# Data Pre-Processing

# Exploring labels

In [ ]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data, sr=sr)
    plt.show()
    
def spectogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(11,4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()

### Fear

In [ ]:
emotion = 'fear'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

### Anger

In [ ]:
emotion = 'angry'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

### Disgust

In [ ]:
emotion = 'disgust'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

### Neutral

In [ ]:
emotion = 'neutral'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

### Sad

In [ ]:
emotion = 'sad'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

### Happy

In [ ]:
emotion = 'sad'
path = np.array(data_path['Path'][data_path['Emotions']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

## Removing Error causing possibilities

In [ ]:
data_path = data_path[data_path['Emotions'] != 'calm']
data_path

In [ ]:
sns.countplot(data=data_path, x='Emotions')

## Shuffling

In [ ]:
data_path = data_path.sample(frac=1)
data_path.head()

In [ ]:
data_path['Emotions'].value_counts()

* Lets get even here... 
* We will use upsampling and downsampling both here
* We will keep 1800 of other emotions of the dataset and increase the number of neutral emotions.
* Let's remove the number of surprises we have for now...


## Deleting till 1800

In [ ]:
# Emotions to reduce
emotions_to_reduce = ['fear', 'angry', 'happy', 'disgust', 'sad']

# Separate out the rows for emotions
reduced_data = (
    data_path[data_path['Emotions'].isin(emotions_to_reduce)]
    .groupby('Emotions')
    .head(1800)  # Keep only the first 1800 samples for these emotions
)

# Keep all samples for emotions not in emotions_to_reduce
other_data = data_path[~data_path['Emotions'].isin(emotions_to_reduce)]

# Combine the reduced emotions and the other emotions
data_path = pd.concat([reduced_data, other_data], ignore_index=True)

In [ ]:
data_path['Emotions'].value_counts()

In [ ]:
data_path.Emotions.describe()

## Removing the Surprises

In [ ]:
data_path = data_path[data_path['Emotions'] != 'surprises']
sns.countplot(data=data_path, x='Emotions')

## Duplicating Neutral Emotion Audio Samples
* We can duplicate a few random audio samples of Neutral Emotion till the count reaches that of the others

In [ ]:
neutral_data = data_path[data_path['Emotions'] == 'neutral']
duplicates_needed = 1800 - len(neutral_data)
neutral_augmented = neutral_data.sample(duplicates_needed, replace=True, random_state=42)

# Combine the original and duplicated 'neutral' data
balanced_neutral_data = pd.concat([neutral_data, neutral_augmented])

# Combine all the balanced data
data_path = pd.concat([reduced_data, balanced_neutral_data])

## Verifying Count

In [ ]:
print(data_path['Emotions'].value_counts())

In [ ]:
sns.countplot(data=data_path, x='Emotions')

# Data Augmentation

In [ ]:
# Function for adding noise to audio
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# Function for stretching the audio
def stretch(data, rate):
    return librosa.effects.time_stretch(data, rate=rate)

# Function for shifting the audio
def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

# Function for adding pitch to audio
def pitch(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor*12)

In [ ]:
# GPU

# def noise(data, noise_factor=0.005):
#     noise = torch.randn_like(data) * noise_factor
#     return data + noise.to("cuda")

# def stretch(data, rate):
#     # Using torchaudio for time stretching
#     return torchaudio.transforms.TimeStretch(rate)(data)

# def shift(data, shift_max=1600):
#     # Implement time shifting using torch.roll
#     shift = torch.randint(-shift_max, shift_max, (1,))
#     return torch.roll(data, int(shift), dims=0)

# def pitch(data, sample_rate, pitch_factor=0):
#     # Using torchaudio for pitch shifting
#     return torchaudio.transforms.PitchShift(sample_rate, pitch_factor)(data)

In [ ]:
# taking any example and checking for the techniques.
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)
# data and sample_rate = diff per path

### 1. Noise Injection

In [ ]:
x = noise(data)
plt.figure(figsize=(14,4))
librosa.display.waveshow(y=x, sr=sample_rate)
Audio(x, rate=sample_rate)

### 2. Stretching
* Changing speed

In [ ]:
x = stretch(data, 0.7)
plt.figure(figsize=(14,4))
librosa.display.waveshow(y=x, sr=sample_rate)
Audio(x, rate=sample_rate)

### 3. Shifting

In [ ]:
x = shift(data)
plt.figure(figsize=(14,4))
librosa.display.waveshow(y=x, sr=sample_rate)
Audio(x, rate=sample_rate)

### 4. Pitch

In [ ]:
x = pitch(data, sample_rate, pitch_factor=np.random.uniform(-5, 5))
plt.figure(figsize=(14,4))
librosa.display.waveshow(y=x, sr=sample_rate)
Audio(x, rate=sample_rate)

In [ ]:
# GPU Code

## Noise
# # Assuming data is a NumPy array
# data_tensor = torch.from_numpy(data)

# # If using GPU
# data_tensor = data_tensor.to("cuda")

# x = noise(data_tensor)

# # Convert back to NumPy array for plotting
# x_numpy = x.cpu().numpy()

# plt.figure(figsize=(14,4))
# librosa.display.waveshow(y=x_numpy, sr=sample_rate)
# Audio(x_numpy, rate=sample_rate)

## Stretching: changing speed
# data_tensor = torch.from_numpy(data).unsqueeze(0)

# # If using GPU
# data_tensor = data_tensor.to("cuda")

# x = stretch(data_tensor)

# x_numpy = x.cpu().numpy()

# plt.figure(figsize=(14,4))
# librosa.display.waveshow(y=x_numpy, sr=sample_rate)
# Audio(x_numpy, rate=sample_rate)

## Shifting
# # Assuming data is a NumPy array
# data_tensor = torch.from_numpy(data).to("cuda")

# # Convert to NumPy (CPU) for `librosa` pitch shifting
# data_numpy = data_tensor.cpu().numpy()
# # Apply pitch shifting
# shifted_data_numpy = librosa.effects.pitch_shift(data_numpy, sr=sample_rate, n_steps=2)

# # Convert back to PyTorch tensor on GPU
# shifted_data_tensor = torch.from_numpy(shifted_data_numpy).to("cuda")

# # Convert back to NumPy for visualization and playback
# x_numpy = shifted_data_tensor.cpu().numpy()

# # Plotting and playback
# plt.figure(figsize=(14, 4))
# librosa.display.waveshow(y=x_numpy, sr=sample_rate)
# plt.title("Pitch Shifted Audio")
# plt.show()
# Audio(x_numpy, rate=sample_rate)

## Pitch
# # Assuming data is a NumPy array
# data_tensor = torch.from_numpy(data).to("cuda")

# # Define the shift amount (positive for right shift, negative for left shift)
# shift_amount = int(0.1 * sample_rate)  # Shift by 0.1 seconds

# # Apply time shifting on GPU
# shifted_data_tensor = torch.roll(data_tensor, shifts=shift_amount)

# # Convert back to NumPy for visualization and playback
# x_numpy = shifted_data_tensor.cpu().numpy()

# # Plotting and playback
# plt.figure(figsize=(14, 4))
# librosa.display.waveshow(y=x_numpy, sr=sample_rate)
# plt.title("Time Shifted Audio")
# plt.show()
# Audio(x_numpy, rate=sample_rate)

# Feature Extraction

In [ ]:
# Using GPU
def extract_features(data, sample_rate):
    result = torch.tensor([]).to('cuda')  # Start with an empty tensor on GPU
    
    # ZCR
    zcr = torch.tensor(librosa.feature.zero_crossing_rate(y=data).mean(axis=1)).to('cuda')
    result = torch.cat((result, zcr), dim=0)  # Concatenate on GPU

    # Chroma_stft
    stft = torch.tensor(np.abs(librosa.stft(data))).to('cuda')
    chroma_stft = torch.tensor(librosa.feature.chroma_stft(S=stft.cpu().numpy(), sr=sample_rate).mean(axis=1)).to('cuda')
    result = torch.cat((result, chroma_stft), dim=0)

    # MFCC
    mfcc = torch.tensor(librosa.feature.mfcc(y=data, sr=sample_rate).mean(axis=1)).to('cuda')
    result = torch.cat((result, mfcc), dim=0)

    # Root Mean Square Value
    rms = torch.tensor(librosa.feature.rms(y=data).mean(axis=1)).to('cuda')
    result = torch.cat((result, rms), dim=0)

    # MelSpectogram
    mel = torch.tensor(librosa.feature.melspectrogram(y=data, sr=sample_rate).mean(axis=1)).to('cuda')
    result = torch.cat((result, mel), dim=0)

    return result.cpu().numpy()  # Convert back to NumPy array if needed

In [ ]:
def augment_and_extract_features(path, n_augment=100):
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # Extract features from the original audio
    original_features = extract_features(data, sample_rate)

    # Randomly choose an augmentation technique
    augmentation_methods = [noise, stretch, shift, pitch]
    method = random.choice(augmentation_methods)

    # Apply the chosen augmentation
    if method == pitch:
        augmented_data = method(data, sample_rate, pitch_factor=np.random.uniform(-5, 5))
    elif method == stretch:
        augmented_data = method(data, rate=np.random.uniform(0.5, 1.5))
    else:
        augmented_data = method(data)

    # Extract features from the augmented audio
    augmented_features = extract_features(augmented_data, sample_rate)

    # Stack original and augmented features vertically
    combined_features = np.vstack((original_features, augmented_features))

    return combined_features

In [ ]:
# GPU

# with tf.device('/device:GPU:0'):
#     # Your model and data operations here
#     # Lists to hold the features and corresponding labels
#     X, Y = [], []

#     # Loop through the dataset
#     for path, emotion in zip(data_path.Path, data_path.Emotions):
#         # Get features for the original and augmented audio
#         features = augment_and_extract_features(path)

#         # Append each set of features to X, and the corresponding emotion to Y
#         for feature in features:
#             X.append(feature)
#             Y.append(emotion)

#     # Convert to numpy arrays
#     X = np.array(X)
#     Y = np.array(Y)

# # Shuffle the dataset
# X, Y = shuffle(X, Y, random_state=42)

In [ ]:
# Dataset processing
X, Y = [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    features = augment_and_extract_features(path)
    for feature in features:
        X.append(feature)
        Y.append(emotion)

    X = np.array(X)
    Y = np.array(Y)

# Shuffle the dataset
X, Y = shuffle(X, Y, random_state=42)

In [ ]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv('features.csv', index=False)
Features.head()

In [ ]:
print(data_path.shape)
data_path.head()

In [ ]:
len(X), len(Y)

In [ ]:
data_path.shape

We have applied data augmentation and extracted the features for each audio files and saved them.

As of now we have extracted the data, now we need to normalize and split our data for training and testing.

In [ ]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

In [ ]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

splitting it in the ratio 3:1

In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
## input split
X = np.expand_dims(X, -1)
X.shape

In [ ]:
enc = OneHotEncoder()
# y = enc.fit_transform(data_path[['label']])
y = enc.fit_transform(data_path[['Emotion']])

In [ ]:
y = y.toarray()
y.shape